In [ ]:
import pandas as pd
import numpy as np

In [ ]:
tab = np.loadtxt('Mortality_from_CVD,_cancer,_diabetes_or_CRD_between_exact_ages_30_and_70_percents.txt')

Bon je galère mdrr j'arrive même pas à charger le fichier correctement mais au moins il est dans le git
Je pense que la problématique ça peut être un truc du genre :
"La mortalité par cancers,CVD, diabètes ou CRD varie-t-elle selon les pays ?"
ou on peut aussi faire un truc du style "La mortalité par cancers,CVD, diabètes ou CRD est-elle plus importante dans les pays avec un système médical moins développé ?" comme ça quand on fera nos clusters on pourra peut-être regarder les pays en fonction de leur système médical.
Mais ça risque d'être plus compliqué à traîter et si on trouve rien de concluant c'est pas fou donc je nte propose qu'on fasse plutôt la première problématique et qu'on fasse un commentaire sur le système médical des pays si on trouve quelque chose d'intéressant à dire dans le clustering.